In [1]:
# Importer des bibliothèques
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import re
from io import StringIO
import os

In [2]:
# Liste d'URLs vers les pages contenant les données de football à extraire
url = ['https://www.kooora.com/?c=1631','https://www.kooora.com/?c=1693',
       'https://www.kooora.com/?c=1578','https://www.kooora.com/?c=1574',
       'https://www.kooora.com/?c=1573','https://www.kooora.com/?c=1438',
       'https://www.kooora.com/?c=1489','https://www.kooora.com/?c=1485',
       'https://www.kooora.com/?c=127','https://www.kooora.com/?c=784',
       'https://www.kooora.com/?c=1433','https://www.kooora.com/?c=2560',
       'https://www.kooora.com/?c=3292','https://www.kooora.com/?c=3968',
       'https://www.kooora.com/?c=4752','https://www.kooora.com/?c=5875',
       'https://www.kooora.com/?c=7607','https://www.kooora.com/?c=8648',
       'https://www.kooora.com/?c=9406','https://www.kooora.com/?c=10693',
       'https://www.kooora.com/?c=10392','https://www.kooora.com/?c=13055',
       'https://www.kooora.com/?c=14595','https://www.kooora.com/?c=16056',
       'https://www.kooora.com/?c=17865','https://www.kooora.com/?c=21404',
       'https://www.kooora.com/?c=22826','https://www.kooora.com/?c=24721']

In [3]:
# Initialisation de la variable de comptage 'i'
i = 0
# Pour le stocker le dataFrame de chaque annee
liste_df_global = []
# Boucle pour extraire et traiter les statistiques de classement à partir des liens URL fournis
for u in url :
    # Faire une requête GET à l'URL u
    page = requests.get(u)
    # Utilisation de BeautifulSoup pour analyser le contenu HTML de la page obtenue précédemment
    soup = BeautifulSoup(page.content, 'html.parser')
    # Recherche de toutes les balises <table> dans le contenu HTML analysé
    tab = soup.find_all("table")
    # Sélection de la deuxième balise <table> de la liste tab
    tab =  tab[1]
    # Trouver la balise de script contenant rankstable (rankstable c'est la table en JS qui contient les données (objectif))
    script_tag = soup.find('script', string=re.compile(r'var rankstable ='))
    # Extraire le contenu du script
    if script_tag:
        script = script_tag.string
    else:
        print("Balise de script introuvable")
    # Définir le motif pour correspondre aux lignes
    motif = re.compile(r'^\s*"r",.*$', re.MULTILINE)
    # Trouver toutes les correspondances en utilisant le motif
    lignes = motif.findall(script)
    # Déclaration d'un liste
    data = []
    # Stocker les lignes correspondantes dans la liste data
    for ligne in lignes :
        data.append(ligne)
    # Créer un flux de texte à partir des données
    data_stream = StringIO("\n".join(data))
    # Lire le flux de texte dans une dataframe pandas
    df = pd.read_csv(data_stream, header=None)
    # Définir les nouveaux noms des colonnes de notre DataFrame
    colonnes = {0:"A" , 1:"B",2:"Classement", 
                3:"C", 4:"Équipe", 5:"Matchs Joués", 
                6:"Victoires", 7:"Égalités", 8:"Défaites", 
                9:"Buts Marqués", 10:"Buts Encaissés", 
                11:"Différence de Buts", 12:"Points", 
                13:"D", 14:"E"}
    # Renommer les colonnes de df
    df.rename(columns=colonnes, inplace=True)
    # Fonction pour extraire le contenu entre les balises <a>
    def extraire_contenu(html):
        soup = BeautifulSoup(html, 'html.parser')
        return soup.get_text()
    # Modifier le contenu des colonnes spécifiées qui contiennent les balises <a>
    colonne_1 = ["Équipe", "Matchs Joués", "Victoires", "Égalités", "Défaites"]
    for colonne in colonne_1 :
        df[colonne] = df[colonne].apply(extraire_contenu)
    # Nettoyer les colonnes spécifiées et le convertir en int
    columns_to_clean = ["Victoires", "Égalités", "Défaites"]
    for col in columns_to_clean:
        df[col] = df[col].apply(lambda x: int(str(x).replace('"', '')))
        # Modifier la colonne 'classement'
    df['Classement'] = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
    # Sélectionner les colonnes utiles et négliger les autres
    colonne = ["Classement", "Équipe", "Matchs Joués", "Victoires", "Égalités", "Défaites", "Buts Marqués", "Buts Encaissés", "Différence de Buts", "Points"]
    df = df[colonne]
    points = []
    # Modifier les lignes commençant par '<nobr><span' dans la colonne 'points'
    for ligne in df['Points'] :
        ch = "<nobr><span"
        if ch in str(ligne) :
            ligne = ligne[-3:-1]
            points.append(ligne)
        else : 
            points.append(ligne)
    df['Points'] = points
    # Convertir les colonnes numériques en int
    col_ = ["Classement", "Matchs Joués", "Victoires", "Égalités", "Défaites", "Buts Marqués", "Buts Encaissés", "Différence de Buts", "Points"]
    for colo in col_:
        df[colo] = df[colo].apply(lambda x: int(str(x)))
    liste_df_global.append(df)
    # Incrémenter l'index pour l'année
    i = i + 1

In [5]:
# Pour enregistrer ces données dans des fichiers Excel
annee = ['1995_1996','1996_1997','1997_1998','1998_1999',
         '1999_2000','2000_2001','2001_2002','2002_2003',
         '2003_2004','2004_2005','2005_2006','2006_2007',
         '2007_2008','2008_2009','2009_2010','2010_2011',
         '2011_2012','2012_2013','2013_2014','2014_2015',
         '2015_2016','2016_2017','2017_2018','2018_2019',
         '2019_2020','2020_2021','2021_2022','2022_2023']
i = 0
chemin = "C:\\Users\\HP 840 G6\\Desktop" #Vous pouvez changer le chemin
for j in liste_df_global :
    j.to_excel(f"{chemin}\\classement_{annee[i]}.xlsx")
    print(f"Le stockage de classement pour l'année {annee[i]} a été effectué avec succès.")
    i = i + 1

Le stockage de classement pour l'année 1995_1996 a été effectué avec succès.
Le stockage de classement pour l'année 1996_1997 a été effectué avec succès.
Le stockage de classement pour l'année 1997_1998 a été effectué avec succès.
Le stockage de classement pour l'année 1998_1999 a été effectué avec succès.
Le stockage de classement pour l'année 1999_2000 a été effectué avec succès.
Le stockage de classement pour l'année 2000_2001 a été effectué avec succès.
Le stockage de classement pour l'année 2001_2002 a été effectué avec succès.
Le stockage de classement pour l'année 2002_2003 a été effectué avec succès.
Le stockage de classement pour l'année 2003_2004 a été effectué avec succès.
Le stockage de classement pour l'année 2004_2005 a été effectué avec succès.
Le stockage de classement pour l'année 2005_2006 a été effectué avec succès.
Le stockage de classement pour l'année 2006_2007 a été effectué avec succès.
Le stockage de classement pour l'année 2007_2008 a été effectué avec succès.